# 1. Enviar os dados para o hdfs

    a. Fazer o download do arquivo no diretório input do SPARK (comandos rodados no terminal Linux)
    
sudo mkdir spark/input/covid
cd covid
sudo curl -O https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/PNI/vacina/uf/2021-11-05/uf%3DSP/part-00000-aa6d7a64-f91d-43b4-afe9-09bc6c16a91e.c000.csv


     b. O arquivo baixado foi enviado para o hdfs (comandos rodados no terminal Linux)
     
hdfs dfs -put /input/covid/part-00000-aa6d7a64-f91d-43b4-afe9-09bc6c16a91e.c000.csv /user/carlos/projeto/

    c. O arquivo foi renomeado para facilitar a codificação do projeto (comandos rodados no terminal Linux)
   
hdfs dfs -mv /user/carlos/data/part-00000-aa6d7a64-f91d-43b4-afe9-09bc6c16a91e.c000.csv /user/carlos/projeto/covid.csv


In [10]:
# Confirmação da cópia e renomeação do arquivo.

!hdfs dfs -ls /user/carlos/projeto/covid.csv

ls: `/user/carlos/projeto/covid.csv': No such file or directory


In [9]:
# Visualização do arquivo.

!hdfs dfs -tail /user/carlos/projeto/covid.csv

5bc1e517b56ccffc2c131f6a3d680739cf4a9c7f8283";"58";"1963-01-23";"M";"99";"SEM INFORMACAO";"355030";"10";"SAO PAULO";"BRASIL";"SP";"";"B";"2028840";"SECRETARIA DE ESTADO DA SAUDE DE SAO PAULO";"INSTITUTO DE INFECTOLOGIA EMILIO RIBAS SAO PAULO";"355030";"SAO PAULO";"SP";"000111";"Outros Imunocomprometidos";"";"Comorbidades";"215VCD129Z";"ASTRAZENECA/FIOCRUZ";"";"2021-08-05";"2ª Dose";"85";"COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD";"VACIVIDA";"2021-08-05T03:08:00.000Z";"18262"
"8655f271-c75b-4aac-9ac4-01a32172376e-i0b0";"7ef5bfcc7a1736da2b7ee9b20037e565306020b486f3e4f5fc9ebb9f65afc725";"63";"1957-08-10";"M";"99";"SEM INFORMACAO";"355030";"10";"SAO PAULO";"BRASIL";"SP";"03514";"B";"2786923";"PREFEITURA DO MUNICIPIO DE SAO PAULO";"UBS CIDADE PATRIARCA DR HERMENEGILDO MORBIN JUNIOR";"355030";"SAO PAULO";"SP";"000201";"Pessoas de 18 a 64 anos";"2";"Faixa Etária";"210183";"ASTRAZENECA/FIOCRUZ";"";"2021-07-28";"2ª Dose";"85";"COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD";"VACIVIDA";"2021-07-28T03

2. Otimizar todos os dados do hdfs para uma tabela Hive particionada por município.

    a. Acessar o serviço hive-server (comandos rodados no terminal Linux)
docker exec -it hive-server bash

    b. Conectar o hive-serve utilizando o Beeline (comandos rodados no terminal Linux):
beeline -u 'jdbc:hive2://localhost:10000 -uroot'

    c. Verificar os databases e criar o database "vacinacao" e a tabela "vacinasp" (comandos rodados no terminal Linux):
show databases;
create database vacinacao;
use vacinacao;
create external table vacinasp
    (
    document_id varchar(50) comment 'ID do Documento do Paciente', 
    paciente_id varchar(60) comment 'ID do Paciente', 
    paciente_idade smallint comment 'Idade do Paciente', 
    paciente_data_nasc string, 
    paciente_enumSexoBiologico varchar(1) comment 'Sexo do Paciente', 
    paciente_raca_cod smallint, 
    paciente_raca_valor string, 
    paciente_cod_municipio int comment 'Codigo do Municipio', 
    paciente_pais int, 
    paciente_municipio string comment 'Nome do Municipio', 
    paciente_pais_nome string, 
    paciente_uf string, 
    paciente_cep string, 
    paciente_nacionalidade string, 
    estabelecimento_valor int, 
    estabelecimento_razao string, 
    estabelecimento_fantasia string, 
    estabelecimento_cod_mun int, 
    estabelecimento_municipio string comment 'Municipio de Aplicacao da Vacina', 
    estabelecimento_uf string, 
    vacina_grupo_cod int comment 'Codigo do Grupo Vacinal', 
    vacina_grupo_nome varchar(30) comment 'Nome do Grupo Vacinal', 
    vacina_categoria_cod int, 
    vacina_categoria_nome string, 
    vacina_lote string, 
    vacina_fabricante_nome varchar(30) comment 'Fabricante da Vacina', 
    vacina_fabricante_ref string, 
    vacina_dataAplicacao date comment 'Data de Aplicacao da Vacina', 
    vacina_descricao_dose string comment 'Numero da Dose', 
    vacina_cod int, 
    vacina_nome varchar(50) comment 'Nome da Vacina', 
    sistema_origem string, 
    time_stamp string, 
    desconhecido string 
    ) 
    row format delimited 
    fields terminated by ';' 
    lines terminated by '\n' 
    location '/user/carlos/projeto/'
    tblproperties('skip.header.line.count'='1')
    ;
    
    d. Criar tabela particionada por municipio (comandos rodados no terminal Linux):

create external table vacinasp_part
    (
    document_id varchar(50) comment 'ID do Documento do Paciente', 
    paciente_id varchar(60) comment 'ID do Paciente', 
    paciente_idade smallint comment 'Idade do Paciente', 
    paciente_enumSexoBiologico varchar(1) comment 'Sexo do Paciente',
    paciente_cod_municipio int comment 'Codigo do Municipio', 
    vacina_grupo_cod int comment 'Codigo do Grupo Vacinal',
    vacina_grupo_nome varchar(30) comment 'Nome do Grupo Vacinal', 
    vacina_fabricante_nome varchar(30) comment 'Fabricante da Vacina', 
    vacina_dataAplicacao date comment 'Data de Aplicacao da Vacina', 
    vacina_descricao_dose string comment 'Numero da Dose',
    vacina_nome varchar(50) comment 'Nome da Vacina'
    ) 
    partitioned by (estabelecimento_municipio string comment 'Municipio de Aplicacao da Vacina')
    location '/user/carlos/projeto/'
    ;
    
    e. Inserir dados (comandos rodados no terminal Linux)
    
insert overwrite table vacinasp_part partition (estabelecimento_municipio) 
    select document_id, 
            paciente_id, 
            paciente_idade, 
            paciente_enumSexoBiologico, 
            paciente_cod_municipio, 
            estabelecimento_municipio,
            vacina_grupo_cod, 
            vacina_grupo_nome, 
            vacina_fabricante_nome, 
            vacina_dataAplicacao, 
            vacina_descricao_dose, 
            vacina_nome 
    from vacinasp;    

3. Criar as 3 vizualizações pelo Spark com os dados enviados para o HDFS

4. Salvar a primeira visualização como tabela Hive
5. Salvar a segunda visualização com formato parquet e compressão snappy
6. Salvar a terceira visualização em um tópico no Kafka
7. Criar a visualização pelo Spark com os dados enviados para o HDFS
8. Salvar a visualização do exercício 6 em um tópico no Elastic
9. Criar um dashboard no Elastic para visualização dos novos dados enviados